## Before submitting
1. Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

2. Make sure that no assertions fail or exceptions occur, otherwise points will be subtracted.

3. After you submit the notebook more tests will be run on your code. The fact that no assertions fail on your computer localy does not guarantee that you completed the exercise correctly.

4. After a function has been tested you can assume that for the following cells a correct implementation is used.

5. Please submit only the edited original `*.ipynb` file.

6. Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Edit only between `YOUR CODE HERE` and `END YOUR CODE`.

7. Make sure to use Python 3, not Python 2.

8. Only work on the exercises using Jupyter Notebook. While tools such as PyCharm and VSCode support the `ipynb` format, they overwrite crucial metadata, which break the autograder system.

9. Do **NOT** under any circustances delete any cells that you didn't insert yourselves. If you accidentally delete a cell either undo the deletion using `Edit/Undo Delete Cells` or redownload the notebook file from ISIS and paste your existing code in there. 


Fill your group name and members below:

In [1]:
GROUPNAME = "Group 56"
COLLABORATORS = "Shiyao Zhao, Ke Wang, Jiajing Jiang"

# Sheet 3: Rounding, Overflow, Linear Algebra

In this exercise sheet, we look at various sources of numerical overflow when executing Python and numpy code for large input values, and how to efficiently handle them, for example, by using numpy special functions.

In [1]:
import utils
import numpy as np

import unittest; t = unittest.TestCase()

In [3]:
# This cell is for grading. Do not delete it

## Building a robust "softplus" nonlinear function (30 P)

The softplus function is defined as:

$$
\mathrm{softplus}(x) = \log(1+\exp(x)).
$$

It intervenes as elementary computation in certain machine learning models such as neural networks. Plotting it gives the following curve

![plot generated using fooplot.com](softplus.png)

where the function tends to zero for very negative input values and tends to the identity for very positive input values.

In [4]:
def softplus(z): 
    return np.log( 1 + np.exp(z))

We consider an input vector from the module `utils` containing varying values between 1 and 10000. We would like to apply the `softplus` function to all of its element in an element-wise manner.

In [5]:
X = utils.softplus_inputs
print(X)

[-10000, -1000, -100, -10, -1, 0, 1, 10, 100, 1000, 10000]


We choose these large values in order to test whether the behavior of the function is correct in all regimes of the function, in particular, for very small or very large values. The code below applies the `softplus` function directly to the vector of inputs and then prints for all cases the input and the corresponding function output:

In [6]:
Y = softplus(X)
for x,y in zip(X,Y):
    print('softplus(%11.4f) = %11.4f'%(x,y))

softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(  1000.0000) =         inf
softplus( 10000.0000) =         inf


For large input values, the softplus function returns `inf` whereas analysis of that function tells us that it should compute the **identity**. Let's now try to apply the softplus function one element at a time, to see whether the problem comes from numpy arrays:

In [7]:
for x in X:
    y = softplus(x)
    print('softplus(%11.4f) = %11.4f'%(x,y))

softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(  1000.0000) =         inf
softplus( 10000.0000) =         inf


Unfortunately, the result is the same. We observe that the function always stops working when its output approaches 1000, even though the input was given in high precision `float64`.

* Create an alternative function for `softplus_robust` that applies to input scalars (int, float, etc.) and that correctly applies to values that can be much larger than 1000 (e.g. billions or more). Your function can be written in Python directly and does not need numpy parallelization.

In [8]:
def softplus_robust(x):
    '''
    Numerically stable implementation of softplus function. Will never overflow to 
    infinity if input is finite
    
    Args:
        x (number): The number of which we want to calculate the softplus value
    Returns:
        number: softplus(x)
    '''
    # YOUR CODE HERE
    #c = x-(x%6)
    return np.log(1+1/np.exp(abs(x))) + np.maximum(x,0)

    raise NotImplementedError("Relplace this line with your code")
    # YOUR CODE HERE
    

In [9]:
# Verify your function
y_scalar = [softplus_robust(x) for x in X]

for x, y in zip(X, y_scalar):
    print('softplus(%11.4f) = %11.4f'%(x,y))


softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(  1000.0000) =   1000.0000
softplus( 10000.0000) =  10000.0000


In [10]:
# This cell is for grading. Do not delete it

As we have seen in the previous exercise sheet, the problem of functions that apply to scalars only is that they are less efficient than functions that apply to vectors directly. Therefore, we would like to handle the rounding issue directly at the vector level.

* Create a new softplus function that applies to vectors and that has the desired behavior for large input values. Your function should be fast for large input vectors (i.e. it is not appropriate to use an inner Python loop inside the function).

**Note**: There are ways to vectorize a function directly (see [`np.vectorize`](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.vectorize.html)/[`np.fromiter`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fromiter.html)/[`np.apply_along_axis`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.apply_along_axis.html)) but those are based on Python loops which provide no speed advantage and should therefore be avoided. It should go without saying, that `for/while` loops and functions like `map` are also not to be used. The cell below should demonstrate how much faster a correct implementation is.

In [11]:
def softplus_robust_vec(X):
    '''
    Vectorized version of the numericaly robust softplus function
    
    Args:
        X (vector-like): A vector (ndim=1) of values on which we want to apply the softplus function.
            It is not always a np.ndarray
        
    Returns:
        np.ndarray: A vector (ndim=1) where the ret[i] == softplus_robust(X[i])
    '''
    # these are wrong!!!
    # return np.array([softplus_robust(x) for x in X])
    # return np.array(list(map(softplus_robust, X)))
    # return np.vectorize(softplus_robust)(X)
    # etc...
    # YOUR CODE HERE
    X = np.array(X)
    return softplus_robust(X)
    raise NotImplementedError("Relplace this line with your code")
    # YOUR CODE HERE
    

In [12]:
# Verify your function
Y = softplus_robust_vec(X)
print(Y)
t.assertIsInstance(Y, np.ndarray)
pairs = tuple(zip(X,Y))
for tup in pairs:
    print('softplus(%11.4f) = %11.4f'%tup)
    
'''
This is just a demonstration.
As long as your vectorized function is consistently faster than the loop implementation,
your solution should be acceptable. 
There are no concrete numbers about the speed-up.
'''
RAND_INPUT = np.random.rand(5000)
print("Vectorized function needs...")
%timeit softplus_robust_vec(RAND_INPUT)

print("Python loops need...")
def vectorize_with_loop(X):
    # This is a wrong implementation
    return np.array([softplus_robust(x) for x in X])
%timeit vectorize_with_loop(RAND_INPUT)


[0.00000000e+00 0.00000000e+00 0.00000000e+00 4.53988992e-05
 3.13261688e-01 6.93147181e-01 1.31326169e+00 1.00000454e+01
 1.00000000e+02 1.00000000e+03 1.00000000e+04]
softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(  1000.0000) =   1000.0000
softplus( 10000.0000) =  10000.0000
Vectorized function needs...
95.3 µs ± 2.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Python loops need...
23.1 ms ± 228 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Computing a partition function (40 P)

We consider a discrete probability distribution of type
$$
p(\boldsymbol{x};\boldsymbol{w}) = \frac{1}{Z(\boldsymbol{w})} \exp(\boldsymbol{x}^\top \boldsymbol{w})
$$
where $\boldsymbol{x} \in \{-1,1\}^{10}$ is an observation, and $\boldsymbol{w} \in \mathbb{R}^{10}$ is a vector of parameters. The term $Z(\boldsymbol{w})$ is called the partition function and is chosen such that the probability distribution sums to 1. That is, the equation:
$$
\sum_{\boldsymbol{x} \in \{-1,1\}^{10}} p(\boldsymbol{x};\boldsymbol{w}) = 1
$$
must be satisfied. Below is a simple method that computes the log of the partition function $Z(\boldsymbol{w})$ for various choices of parameter vectors. The considered parameters (`w_small`, `w_medium`, and `w_large`) are increasingly large (and thus problematic), and can be found in the file `utils.py`.

In [13]:
import itertools

def iter_all_observations():
    '''
    Iterates over all x in {-1,1}^10 (vectors with 10 elements where each element 
    containts either -1 or 1)
    
    Returns:
        iterator : An iterator of all valid obvervations
    '''
    return itertools.product([-1, 1], repeat=10)

def getlogZ(w):
    '''
    Calculates the log of the partition function Z
    
    Args:
        w (vector-like): A ten element vector (shape=(10,)) of parameters
    Returns:
        number: The log of the partition function Z
    '''
    Z = sum(np.exp(np.dot(x,w)) for x in iter_all_observations())
    #print(Z)
    return np.log(Z)

print('%11.4f'%getlogZ(utils.w_small))
print('%11.4f'%getlogZ(utils.w_medium))
print('%11.4f'%getlogZ(utils.w_big))


    17.2457
    89.5932
        inf


We can observe from these results, that for parameter vectors with large values (e.g. `utils.w_big`), the exponential function overflows, and thus, we do not obtain a correct value for the logarithm of `Z`.

* Implement an improved function  `getlogZ_robust` that avoids the overflow problem, and evaluates the partition function for the same parameters.

In [14]:
def getlogZ_robust(w):
    # YOUR CODE HERE
    c = max(np.dot(x,w) for x in iter_all_observations())
    Z = sum(np.exp(np.dot(x,w)-c) for x in iter_all_observations())
    return np.log(Z)+c
    raise NotImplementedError("Relplace this line with your code")
    # YOUR CODE HERE
    

In [15]:
# Verify your function
#print(utils.w_medium)
print(utils.w_big)
print('%11.4f'%getlogZ_robust(utils.w_small))
print('%11.4f'%getlogZ_robust(utils.w_medium))
print('%11.4f'%getlogZ_robust(utils.w_big))

R = getlogZ_robust(utils.w_big)
t.assertTrue(np.isfinite(R))
t.assertTrue(24900 < R < 25000)

[-1.52820754859, -0.000234000845064, -0.00527938881237, 5797.19232191, -6.64682108484, 18924.7087966, -69.308158911, 1.1158892974, 1.04454511882, 116.795573742]
    17.2457
    89.5932
 24919.9913


In [16]:
# This cell is for grading. Do not delete it

* For the model with parameter `utils.w_big`, evaluate the log-probability of the binary vectors contained in the list `itertools.product([-1, 1], repeat=10)`, and return a `np.ndarray` of the indices (starting from 0) of those that have probability greater or equal to 0.001.

In [17]:
def important_indexes(tol = 0.001):
    '''
    Calculates the indexes of important binary vectors for the w_big parameter vector.
    
    Args:
        tol (float): The thesh
    '''
    Z = getlogZ_robust(utils.w_big)

    # YOUR CODE HERE
    p = list()
    for x in iter_all_observations():
        p.append(np.exp(np.dot(x,utils.w_big)-Z))
    
    return np.array([index for index,value in enumerate(p) if value>=tol])
    raise NotImplementedError("Relplace this line with your code")
    # YOUR CODE HERE
    

In [18]:
# Verify your function
print(important_indexes())
t.assertEqual(len(important_indexes()), 24)

[ 81  83  85  87 209 211 213 215 337 339 341 343 465 467 469 471 597 599
 725 727 853 855 981 983]


## Probability of generating data from a Gaussian model (30 P)

Consider a multivariate Gaussian distribution of mean vector `m` and covariance `S`. The probability associated to a vector `x` is given by:

$$
p(\boldsymbol{x};(\boldsymbol{m},S)) = \frac{1}{\sqrt{(2\pi)^d \mathrm{det}(S)}} \exp \Big( - \frac12 (\boldsymbol{x}-\boldsymbol{m})^\top S^{-1} (\boldsymbol{x}-\boldsymbol{m})\Big)
$$

We consider the calculation of the probability of observing a certain dataset 

$$
\mathcal{D} = (\boldsymbol{x}^{(1)},\dots,\boldsymbol{x}^{(N)})
$$

assuming the data is generated according to a Gaussian distribution of fixed parameters $\boldsymbol{m}$ and $S$. Such probability density is given by the formula:

$$
\log P(\mathcal{D};(\boldsymbol{m},S)) = \log \prod_{i=1}^N p(\boldsymbol{x}^{(i)};(\boldsymbol{m},S))
$$

The function below implements such function:

In [19]:
def logp(X,m,S):
    # Find the number of dimensions from the data vector
    d = X.shape[1]
    
    # Invert the covariance matrix
    Sinv = np.linalg.inv(S)
    
    # Compute the quadratic terms for all data points
    Q = -0.5*(np.dot(X-m,Sinv)*(X-m)).sum(axis=1)
    
    # Raise them quadratic terms to the exponential
    Q = np.exp(Q)
    
    # Divide by the terms in the denominator
    P = Q / np.sqrt((2*np.pi)**d * np.linalg.det(S))
    
    # Take the product of the probability of each data points
    Pprod = np.prod(P)
    
    # Return the log-probability
    return np.log(Pprod)


Evaluation of this function for various datasets and parameters provided in the file `utils.py` gives the following probabilities:

In [20]:
print('%11.4f'%logp(utils.X1,utils.m1,utils.S1))
print('%11.4f'%logp(utils.X2,utils.m2,utils.S2))
print('%11.4f'%logp(utils.X3,utils.m3,utils.S3))

   -13.0068
       -inf
       -inf


/Users/shiyaozhao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log


This function is numerically unstable for multiple reasons. The product of many probabilities, the inversion of a large covariance matrix, and the computation of its determinant, are all potential causes for overflow. Thus, we would like to find a numerically robust way of performing each of these.

* Implement a numerically stable version of the function `logp`
* Evaluate it on the same datasets and parameters as the function `logp`

In [23]:
def logp_robust(X,m,S):
    '''
    Numerically robust implemenation of `logp` function
    
    Returns:
        (float): The logp probability density
    '''
    # YOUR CODE HERE
    d = m.shape[0]

    # Invert the covariance matrix
    Sinv = np.linalg.inv(S)
    
    # Compute the quadratic terms for all data points
    Q = -0.5*(np.dot(X-m,Sinv)*(X-m)).sum(axis=1)
    
    # compute the denominator
    denominator =  np.sqrt((2*np.pi)**d * np.linalg.det(S))
    
    # compute the log-probability
    log_P = Q-np.log(denominator)
    
    # Return the sum of log-probability
    return np.sum(log_P)
    
    raise NotImplementedError("Relplace this line with your code")
    # YOUR CODE HERE
    

In [24]:
# Verify your function
print('%11.4f'%logp_robust(utils.X1,utils.m1,utils.S1))
print('%11.4f'%logp_robust(utils.X2,utils.m2,utils.S2))
print('%11.4f'%logp_robust(utils.X3,utils.m3,utils.S3))

logp1 = logp_robust(utils.X1,utils.m1,utils.S1)
logp2 = logp_robust(utils.X2,utils.m2,utils.S2)
logp3 = logp_robust(utils.X3,utils.m3,utils.S3)

outputs = np.array((logp1, logp2, logp3))

t.assertTrue(np.isfinite(outputs).all())
t.assertTrue(np.all(outputs  < 0))
print('\nall logp values below zero :)')

   -13.0068
 -1947.9710
-218646.1739

all logp values below zero :)
